# Column and Row Key Explorations

In [1]:
import polars as pl
import polars.selectors as cs
from glob import glob
from toolz import pipe
from functools import reduce
from columns import activity_columns,  roster_columns
from operator import add, mul




## conduct row key evaluation

### Summary of Row Key join Evaluation
- It appears that all keys exist in both unioned data sets as we can see by searching for any case id that doesn't perfectly  match in both tables.

In [2]:
#This is used as part of my temporary union below.
join_next = lambda df1, df2: (df1.join(df2, on='Column', how = 'full', suffix = '_right')
                                 .with_columns(Column = pl.coalesce('Column', 'Column_right'))
                                 .drop('Column_right')
                             )

### Temp Union Data

- code in this section is intended to act as a join key exploration, Its design is to allow users to examine any mismatches between the key's in different sheets.
- this code is meant as an example and did not affect the final product.

#### ACt Union

In [3]:
(act_paths :=
 glob('./data/atusact*.dat') + glob('./data/*/atusact*.dat')
)

['./data/atusact_2003.dat',
 './data/atusact_2004.dat',
 './data/atusact_2005.dat',
 './data/atusact_2006.dat',
 './data/atusact_2007.dat',
 './data/atusact_2008.dat',
 './data/atusact_2009.dat',
 './data/atusact_2010.dat',
 './data/atusact_2011.dat',
 './data/atusact_2012.dat',
 './data/atusact_2013.dat',
 './data/atusact_2014.dat',
 './data/atusact_2015.dat',
 './data/atusact_2017.dat',
 './data/atusact_2018.dat',
 './data/atusact_2019.dat',
 './data/atusact_2020.dat',
 './data/atusact_2021.dat',
 './data/atusact_2022.dat',
 './data/atusact_2023.dat',
 './data/atusact_2024.dat',
 './data/atusact_2016/atusact_2016.dat']

In [4]:
(original_columns :=
 {p:{col: col
     for col in 
     pl.read_csv(p).columns
    }
  for p in act_paths
 }
)

{'./data/atusact_2003.dat': {'TUCASEID': 'TUCASEID',
  'TUACTDUR24': 'TUACTDUR24',
  'TRTCC_LN': 'TRTCC_LN',
  'TRTCOC_LN': 'TRTCOC_LN',
  'TUACTIVITY_N': 'TUACTIVITY_N',
  'TUTIER1CODE': 'TUTIER1CODE',
  'TUTIER2CODE': 'TUTIER2CODE',
  'TUTIER3CODE': 'TUTIER3CODE',
  'TUSTARTTIM': 'TUSTARTTIM',
  'TUSTOPTIME': 'TUSTOPTIME',
  'TUACTDUR': 'TUACTDUR',
  'TUCC8': 'TUCC8',
  'TUCUMDUR': 'TUCUMDUR',
  'TEWHERE': 'TEWHERE',
  'TXWHERE': 'TXWHERE',
  'TR_03CC57': 'TR_03CC57',
  'TUCUMDUR24': 'TUCUMDUR24'},
 './data/atusact_2004.dat': {'TUCASEID': 'TUCASEID',
  'TUACTIVITY_N': 'TUACTIVITY_N',
  'TUTIER1CODE': 'TUTIER1CODE',
  'TUTIER2CODE': 'TUTIER2CODE',
  'TUTIER3CODE': 'TUTIER3CODE',
  'TUSTARTTIM': 'TUSTARTTIM',
  'TUSTOPTIME': 'TUSTOPTIME',
  'TUACTDUR': 'TUACTDUR',
  'TUCUMDUR': 'TUCUMDUR',
  'TUCC5': 'TUCC5',
  'TUCC7': 'TUCC7',
  'TUCC8': 'TUCC8',
  'TEWHERE': 'TEWHERE',
  'TXWHERE': 'TXWHERE',
  'TUCC5B': 'TUCC5B',
  'TUACTDUR24': 'TUACTDUR24',
  'TUCUMDUR24': 'TUCUMDUR24',
  'TRTCC_

In [5]:
(fixed_columns :=
 {**original_columns, 
  **{
      path:{col:col.upper() for col in original_columns.get(path,[])}
     for path in act_paths
     },  
}
)

{'./data/atusact_2003.dat': {'TUCASEID': 'TUCASEID',
  'TUACTDUR24': 'TUACTDUR24',
  'TRTCC_LN': 'TRTCC_LN',
  'TRTCOC_LN': 'TRTCOC_LN',
  'TUACTIVITY_N': 'TUACTIVITY_N',
  'TUTIER1CODE': 'TUTIER1CODE',
  'TUTIER2CODE': 'TUTIER2CODE',
  'TUTIER3CODE': 'TUTIER3CODE',
  'TUSTARTTIM': 'TUSTARTTIM',
  'TUSTOPTIME': 'TUSTOPTIME',
  'TUACTDUR': 'TUACTDUR',
  'TUCC8': 'TUCC8',
  'TUCUMDUR': 'TUCUMDUR',
  'TEWHERE': 'TEWHERE',
  'TXWHERE': 'TXWHERE',
  'TR_03CC57': 'TR_03CC57',
  'TUCUMDUR24': 'TUCUMDUR24'},
 './data/atusact_2004.dat': {'TUCASEID': 'TUCASEID',
  'TUACTIVITY_N': 'TUACTIVITY_N',
  'TUTIER1CODE': 'TUTIER1CODE',
  'TUTIER2CODE': 'TUTIER2CODE',
  'TUTIER3CODE': 'TUTIER3CODE',
  'TUSTARTTIM': 'TUSTARTTIM',
  'TUSTOPTIME': 'TUSTOPTIME',
  'TUACTDUR': 'TUACTDUR',
  'TUCUMDUR': 'TUCUMDUR',
  'TUCC5': 'TUCC5',
  'TUCC7': 'TUCC7',
  'TUCC8': 'TUCC8',
  'TEWHERE': 'TEWHERE',
  'TXWHERE': 'TXWHERE',
  'TUCC5B': 'TUCC5B',
  'TUACTDUR24': 'TUACTDUR24',
  'TUCUMDUR24': 'TUCUMDUR24',
  'TRTCC_

In [6]:
(example_correct_table := pl.read_csv(act_paths[0]).head()
)

TUCASEID,TUACTDUR24,TRTCC_LN,TRTCOC_LN,TUACTIVITY_N,TUTIER1CODE,TUTIER2CODE,TUTIER3CODE,TUSTARTTIM,TUSTOPTIME,TUACTDUR,TUCC8,TUCUMDUR,TEWHERE,TXWHERE,TR_03CC57,TUCUMDUR24
i64,i64,i64,i64,i64,i64,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64
20030100013280,60,-1,-1,1,13,1,24,"""04:00:00""","""05:00:00""",60,97,60,9,0,-1,60
20030100013280,30,-1,-1,2,1,2,1,"""05:00:00""","""05:30:00""",30,0,90,-1,0,-1,90
20030100013280,600,-1,-1,3,1,1,1,"""05:30:00""","""15:30:00""",600,0,690,-1,0,-1,690
20030100013280,150,-1,-1,4,12,3,3,"""15:30:00""","""18:00:00""",150,0,840,1,0,-1,840
20030100013280,5,-1,-1,5,11,1,1,"""18:00:00""","""18:05:00""",5,0,845,1,0,-1,845


In [7]:
(int_columns := example_correct_table.select(cs.integer()).columns
)

['TUCASEID',
 'TUACTDUR24',
 'TRTCC_LN',
 'TRTCOC_LN',
 'TUACTIVITY_N',
 'TUTIER1CODE',
 'TUTIER2CODE',
 'TUTIER3CODE',
 'TUACTDUR',
 'TUCC8',
 'TUCUMDUR',
 'TEWHERE',
 'TXWHERE',
 'TR_03CC57',
 'TUCUMDUR24']

In [8]:
(str_columns := example_correct_table.select(cs.string()).columns
)

['TUSTARTTIM', 'TUSTOPTIME']

In [9]:
(col_and_types := {c: pl.String() for c in str_columns
} |    # Merge operator
{c:pl.Int64() for c in int_columns
 if c in activity_columns #pl.read_csv(p,n_rows=100).columns  # only cast columns that we need
        
}
 )

{'TUSTARTTIM': String,
 'TUSTOPTIME': String,
 'TUCASEID': Int64,
 'TUACTDUR24': Int64,
 'TUACTIVITY_N': Int64,
 'TUTIER1CODE': Int64,
 'TUTIER2CODE': Int64,
 'TUTIER3CODE': Int64}

In [10]:
(dfs := [pl.read_csv(p)
       .rename(col_rename)
       .select([
            pl.col(c).cast(t)
            for c, t in col_and_types.items()
            if c in activity_columns  # only cast columns that exist
        ]) for p, col_rename in fixed_columns.items()])

[shape: (412_611, 6)
 ┌────────────────┬────────────┬──────────────┬─────────────┬─────────────┬─────────────┐
 │ TUCASEID       ┆ TUACTDUR24 ┆ TUACTIVITY_N ┆ TUTIER1CODE ┆ TUTIER2CODE ┆ TUTIER3CODE │
 │ ---            ┆ ---        ┆ ---          ┆ ---         ┆ ---         ┆ ---         │
 │ i64            ┆ i64        ┆ i64          ┆ i64         ┆ i64         ┆ i64         │
 ╞════════════════╪════════════╪══════════════╪═════════════╪═════════════╪═════════════╡
 │ 20030100013280 ┆ 60         ┆ 1            ┆ 13          ┆ 1           ┆ 24          │
 │ 20030100013280 ┆ 30         ┆ 2            ┆ 1           ┆ 2           ┆ 1           │
 │ 20030100013280 ┆ 600        ┆ 3            ┆ 1           ┆ 1           ┆ 1           │
 │ 20030100013280 ┆ 150        ┆ 4            ┆ 12          ┆ 3           ┆ 3           │
 │ 20030100013280 ┆ 5          ┆ 5            ┆ 11          ┆ 1           ┆ 1           │
 │ …              ┆ …          ┆ …            ┆ …           ┆ …           ┆ …  

In [11]:
(common_cols := set(dfs[0].columns))
for df in dfs[1:]:
    common_cols &= set(df.columns)


In [12]:
(act_csv := pl.concat([df.select(sorted(common_cols)) for df in dfs]))

TUACTDUR24,TUACTIVITY_N,TUCASEID,TUTIER1CODE,TUTIER2CODE,TUTIER3CODE
i64,i64,i64,i64,i64,i64
60,1,20030100013280,13,1,24
30,2,20030100013280,1,2,1
600,3,20030100013280,1,1,1
150,4,20030100013280,12,3,3
5,5,20030100013280,11,1,1
…,…,…,…,…,…
120,7,20161212162509,12,1,1
20,8,20161212162509,11,1,1
130,9,20161212162509,12,3,7


#### Rost union

In [13]:
(rost_paths :=
 glob('./data/atusrost*.dat') + glob('./data/*/atusrost*.dat')
)

['./data/atusrost_2003.dat',
 './data/atusrost_2004.dat',
 './data/atusrost_2005.dat',
 './data/atusrost_2006.dat',
 './data/atusrost_2007.dat',
 './data/atusrost_2008.dat',
 './data/atusrost_2009.dat',
 './data/atusrost_2010.dat',
 './data/atusrost_2011.dat',
 './data/atusrost_2012.dat',
 './data/atusrost_2013.dat',
 './data/atusrost_2014.dat',
 './data/atusrost_2015.dat',
 './data/atusrost_2017.dat',
 './data/atusrost_2018.dat',
 './data/atusrost_2019.dat',
 './data/atusrost_2020.dat',
 './data/atusrost_2021.dat',
 './data/atusrost_2022.dat',
 './data/atusrost_2023.dat',
 './data/atusrost_2024.dat',
 './data/atusrost_2016/atusrost_2016.dat']

In [14]:
(original_columns :=
 {p:{col: col
     for col in 
     pl.read_csv(p).columns
    }
  for p in rost_paths
 }
)

{'./data/atusrost_2003.dat': {'TUCASEID': 'TUCASEID',
  'TULINENO': 'TULINENO',
  'TURRP': 'TURRP',
  'TESEX': 'TESEX',
  'TERRP': 'TERRP',
  'TEAGE': 'TEAGE',
  'TXSEX': 'TXSEX',
  'TXRRP': 'TXRRP',
  'TXAGE': 'TXAGE'},
 './data/atusrost_2004.dat': {'TUCASEID': 'TUCASEID',
  'TULINENO': 'TULINENO',
  'TURRP': 'TURRP',
  'TESEX': 'TESEX',
  'TERRP': 'TERRP',
  'TEAGE': 'TEAGE',
  'TXSEX': 'TXSEX',
  'TXRRP': 'TXRRP',
  'TXAGE': 'TXAGE'},
 './data/atusrost_2005.dat': {'TUCASEID': 'TUCASEID',
  'TULINENO': 'TULINENO',
  'TEAGE': 'TEAGE',
  'TERRP': 'TERRP',
  'TESEX': 'TESEX',
  'TXAGE': 'TXAGE',
  'TXRRP': 'TXRRP',
  'TXSEX': 'TXSEX'},
 './data/atusrost_2006.dat': {'TUCASEID': 'TUCASEID',
  'TULINENO': 'TULINENO',
  'TEAGE': 'TEAGE',
  'TERRP': 'TERRP',
  'TESEX': 'TESEX',
  'TXAGE': 'TXAGE',
  'TXRRP': 'TXRRP',
  'TXSEX': 'TXSEX'},
 './data/atusrost_2007.dat': {'TUCASEID': 'TUCASEID',
  'TULINENO': 'TULINENO',
  'TEAGE': 'TEAGE',
  'TERRP': 'TERRP',
  'TESEX': 'TESEX',
  'TXAGE': 'TXAG

In [15]:
(fixed_columns :=
 {**original_columns, 
  **{
      path:{col:col.upper() for col in original_columns.get(path,[])}
     for path in rost_paths
     },  
}
)

{'./data/atusrost_2003.dat': {'TUCASEID': 'TUCASEID',
  'TULINENO': 'TULINENO',
  'TURRP': 'TURRP',
  'TESEX': 'TESEX',
  'TERRP': 'TERRP',
  'TEAGE': 'TEAGE',
  'TXSEX': 'TXSEX',
  'TXRRP': 'TXRRP',
  'TXAGE': 'TXAGE'},
 './data/atusrost_2004.dat': {'TUCASEID': 'TUCASEID',
  'TULINENO': 'TULINENO',
  'TURRP': 'TURRP',
  'TESEX': 'TESEX',
  'TERRP': 'TERRP',
  'TEAGE': 'TEAGE',
  'TXSEX': 'TXSEX',
  'TXRRP': 'TXRRP',
  'TXAGE': 'TXAGE'},
 './data/atusrost_2005.dat': {'TUCASEID': 'TUCASEID',
  'TULINENO': 'TULINENO',
  'TEAGE': 'TEAGE',
  'TERRP': 'TERRP',
  'TESEX': 'TESEX',
  'TXAGE': 'TXAGE',
  'TXRRP': 'TXRRP',
  'TXSEX': 'TXSEX'},
 './data/atusrost_2006.dat': {'TUCASEID': 'TUCASEID',
  'TULINENO': 'TULINENO',
  'TEAGE': 'TEAGE',
  'TERRP': 'TERRP',
  'TESEX': 'TESEX',
  'TXAGE': 'TXAGE',
  'TXRRP': 'TXRRP',
  'TXSEX': 'TXSEX'},
 './data/atusrost_2007.dat': {'TUCASEID': 'TUCASEID',
  'TULINENO': 'TULINENO',
  'TEAGE': 'TEAGE',
  'TERRP': 'TERRP',
  'TESEX': 'TESEX',
  'TXAGE': 'TXAG

In [16]:
(example_correct_table := pl.read_csv(rost_paths[0]).head()
)

TUCASEID,TULINENO,TURRP,TESEX,TERRP,TEAGE,TXSEX,TXRRP,TXAGE
i64,i64,i64,i64,i64,i64,i64,i64,i64
20030100013280,1,18,1,18,60,0,0,0
20030100013280,2,20,2,20,72,0,0,0
20030100013280,3,22,2,22,37,0,0,0
20030100013344,1,18,2,18,41,0,0,0
20030100013344,2,20,1,20,42,0,0,0


In [17]:
(int_columns := example_correct_table.select(cs.integer()).columns
)

['TUCASEID',
 'TULINENO',
 'TURRP',
 'TESEX',
 'TERRP',
 'TEAGE',
 'TXSEX',
 'TXRRP',
 'TXAGE']

In [18]:
(str_columns := example_correct_table.select(cs.string()).columns
)

[]

In [19]:
(col_and_types := {c: pl.String() for c in str_columns
} |    # Merge operator
{c:pl.Int64() for c in int_columns
 if c in roster_columns #pl.read_csv(p,n_rows=100).columns  # only cast columns that we need
        
}
 )

{'TUCASEID': Int64,
 'TULINENO': Int64,
 'TESEX': Int64,
 'TERRP': Int64,
 'TEAGE': Int64}

In [20]:
(dfs := [pl.read_csv(p)
       .rename(col_rename)
       .select([
            pl.col(c).cast(t)
            for c, t in col_and_types.items()
            if c in roster_columns # only cast columns that exist
        ]) for p, col_rename in fixed_columns.items()])

[shape: (58_911, 5)
 ┌────────────────┬──────────┬───────┬───────┬───────┐
 │ TUCASEID       ┆ TULINENO ┆ TESEX ┆ TERRP ┆ TEAGE │
 │ ---            ┆ ---      ┆ ---   ┆ ---   ┆ ---   │
 │ i64            ┆ i64      ┆ i64   ┆ i64   ┆ i64   │
 ╞════════════════╪══════════╪═══════╪═══════╪═══════╡
 │ 20030100013280 ┆ 1        ┆ 1     ┆ 18    ┆ 60    │
 │ 20030100013280 ┆ 2        ┆ 2     ┆ 20    ┆ 72    │
 │ 20030100013280 ┆ 3        ┆ 2     ┆ 22    ┆ 37    │
 │ 20030100013344 ┆ 1        ┆ 2     ┆ 18    ┆ 41    │
 │ 20030100013344 ┆ 2        ┆ 1     ┆ 20    ┆ 42    │
 │ …              ┆ …        ┆ …     ┆ …     ┆ …     │
 │ 20031212033636 ┆ 2        ┆ 2     ┆ 20    ┆ 47    │
 │ 20031212033636 ┆ 3        ┆ 2     ┆ 22    ┆ 12    │
 │ 20031212033636 ┆ 4        ┆ 1     ┆ 22    ┆ 13    │
 │ 20031212033642 ┆ 1        ┆ 2     ┆ 18    ┆ 54    │
 │ 20031212033642 ┆ 2        ┆ 1     ┆ 20    ┆ 56    │
 └────────────────┴──────────┴───────┴───────┴───────┘,
 shape: (39_401, 5)
 ┌────────────────┬─────

In [21]:
# find common columns
(common_cols := set(dfs[0].columns))
for df in dfs[1:]:
    common_cols &= set(df.columns)


In [22]:
(rost_csv := pl.concat([df.select(sorted(common_cols)) for df in dfs]))

TEAGE,TERRP,TESEX,TUCASEID,TULINENO
i64,i64,i64,i64,i64
60,18,1,20030100013280,1
72,20,2,20030100013280,2
37,22,2,20030100013280,3
41,18,2,20030100013344,1
42,20,1,20030100013344,2
…,…,…,…,…
80,19,2,20161212162456,1
16,18,2,20161212162509,1
37,24,1,20161212162509,2


### Row Key Evaluation

In [23]:
(info_keys :=
    act_csv
   .group_by('TUCASEID')
   .len()
   .with_columns(pl.lit(1).alias('act_csv'))
   .drop('len')
)


TUCASEID,act_csv
i64,i32
20220807222422,1
20161109161898,1
20190302191489,1
20080807082481,1
20130807131128,1
…,…
20030403032895,1
20090403091347,1
20230706230269,1


In [24]:

(rost_keys := (
    rost_csv
    .group_by('TUCASEID')
    .len()
    .with_columns(pl.lit(1).alias("rost_csv"))
    .drop('len')
)
)

TUCASEID,rost_csv
i64,i32
20050302052453,1
20100706102515,1
20141110142019,1
20031211030679,1
20060605061501,1
…,…
20200201200399,1
20211211211248,1
20220605222094,1


In [25]:



(key_summary := (
    info_keys
    .join(rost_keys, on="TUCASEID", how="full")
    .fill_null(0)
    .with_columns(
        all = pl.min_horizontal(cs.ends_with("csv")),
        any = pl.max_horizontal(cs.ends_with("csv")),
        count = pl.sum_horizontal(cs.ends_with("csv")),
    )
)
)


TUCASEID,act_csv,TUCASEID_right,rost_csv,all,any,count
i64,i32,i64,i32,i32,i32,i32
20050302052453,1,20050302052453,1,1,1,2
20100706102515,1,20100706102515,1,1,1,2
20141110142019,1,20141110142019,1,1,1,2
20031211030679,1,20031211030679,1,1,1,2
20060605061501,1,20060605061501,1,1,1,2
…,…,…,…,…,…,…
20200201200399,1,20200201200399,1,1,1,2
20211211211248,1,20211211211248,1,1,1,2
20220605222094,1,20220605222094,1,1,1,2


In [26]:
key_summary.filter(pl.col('all') ==0)

TUCASEID,act_csv,TUCASEID_right,rost_csv,all,any,count
i64,i32,i64,i32,i32,i32,i32


## TEST UNION COMPATIBILITY
 - The below lambda is used to do  the actual specific type of join that gets the overall column exploration in the f ormat we are using.
 - To ensure that the columns needed are in all of the data years that need to be unioned together for each union type we will...
     -  grab the relative file paths with glob.
     -  get a list of all of the distinct column names across all of the many files that need to be unioned, and unpivot those column names to make our rows.
     -  Next we will set the name of each .dat file as the column name.
     -  The  logic that returns the  ones for each column join exploration is that if a .dat file contains a table that has a given file name in the overall list of possible column names. Then it will have a 1. If it does not contain that column name,  it will be a 0.
         -  If all tables contain a given  column name, then all column will = 1.
         -  the count column will sum all of the ones to show how many total files contain a given column name.
    - this will be done across the 4 file types.  

### summary of column union compatibility
 - while there are many mismatches in column names across the different years...
     -  We don't actually need all of the data and for the most part these mismatches were harmless.
     -  However, in the respondent files there was some issues that BLS noted  about the TUFINLWGT column that will require some extra work for a few reasons.
         - TUFINLWGT does not exist in all years.
         - in 2003-2006 the data in that column was not accurate, so we need to handle that specially.
    - This will be handled specially inside of the unioning in the project 1-3 portion of this code.  

In [27]:
 # this fx is the same for all cells so no reason to have it copied many times
join_next = lambda df1, df2: (df1.join(df2, on='Column', how = 'full', suffix = '_right')
                                 .with_columns(Column = pl.coalesce('Column', 'Column_right'))
                                 .drop('Column_right')
                             )

### ACTIVITY PRE-UNION COLUMN TEST

In [28]:
type = 'atusact'
(act_paths :=
 glob('./data/atusact*.dat') + glob('./data/*/atusact*.dat')
)


['./data/atusact_2003.dat',
 './data/atusact_2004.dat',
 './data/atusact_2005.dat',
 './data/atusact_2006.dat',
 './data/atusact_2007.dat',
 './data/atusact_2008.dat',
 './data/atusact_2009.dat',
 './data/atusact_2010.dat',
 './data/atusact_2011.dat',
 './data/atusact_2012.dat',
 './data/atusact_2013.dat',
 './data/atusact_2014.dat',
 './data/atusact_2015.dat',
 './data/atusact_2017.dat',
 './data/atusact_2018.dat',
 './data/atusact_2019.dat',
 './data/atusact_2020.dat',
 './data/atusact_2021.dat',
 './data/atusact_2022.dat',
 './data/atusact_2023.dat',
 './data/atusact_2024.dat',
 './data/atusact_2016/atusact_2016.dat']

In [29]:
(act_column_summary :=
 pipe(act_paths,
     lambda L: [pl.read_csv(p)
                  .with_columns(file = pl.lit(p.replace('\\','/').split('/')[-1]))
                  .head(1) 
                  .unpivot(index='file', 
                           variable_name='Column')
                  .drop('value')
                  .with_columns(pl.lit(1).alias('ones'))
                  .pivot(index = 'Column', 
                         on='file', 
                         values='ones') 
                for p in L 
                ],
     lambda L: reduce(join_next, L).sort('Column').fill_null(0),
     lambda df: df.with_columns(all = pl.reduce(mul, cs.starts_with(type)), 
                                count = pl.reduce(add, cs.starts_with(type))
                               ),
     )
)

Column,atusact_2003.dat,atusact_2004.dat,atusact_2005.dat,atusact_2006.dat,atusact_2007.dat,atusact_2008.dat,atusact_2009.dat,atusact_2010.dat,atusact_2011.dat,atusact_2012.dat,atusact_2013.dat,atusact_2014.dat,atusact_2015.dat,atusact_2017.dat,atusact_2018.dat,atusact_2019.dat,atusact_2020.dat,atusact_2021.dat,atusact_2022.dat,atusact_2023.dat,atusact_2024.dat,atusact_2016.dat,all,count
str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""TEWHERE""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22
"""TRCODE""",0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,18
"""TRTCCTOT_LN""",0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,21
"""TRTCC_LN""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22
"""TRTCOC_LN""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""TUSTOPTIME""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22
"""TUTIER1CODE""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22
"""TUTIER2CODE""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22


In [30]:
#  show  only columns that are mismatched between the different years.
# in effect none of the columns here are needed so we will allow alll of them to be automatically removed in the next part of the project when unioning

act_column_summary.filter(pl.col('all')==0)

Column,atusact_2003.dat,atusact_2004.dat,atusact_2005.dat,atusact_2006.dat,atusact_2007.dat,atusact_2008.dat,atusact_2009.dat,atusact_2010.dat,atusact_2011.dat,atusact_2012.dat,atusact_2013.dat,atusact_2014.dat,atusact_2015.dat,atusact_2017.dat,atusact_2018.dat,atusact_2019.dat,atusact_2020.dat,atusact_2021.dat,atusact_2022.dat,atusact_2023.dat,atusact_2024.dat,atusact_2016.dat,all,count
str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""TRCODE""",0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,18
"""TRTCCTOT_LN""",0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,21
"""TRTEC_LN""",0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,14
"""TRTHH_LN""",0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,21
"""TRTIER2""",0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,18
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""TUCC5""",0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,21
"""TUCC5B""",0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,21
"""TUCC7""",0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,21


### WHO PRE-UNION COLUMN TEST

In [31]:
type = 'atuswho'
(who_paths :=
 glob('./data/atuswho*.dat') + glob('./data/*/atuswho*.dat')
)

['./data/atuswho_2004.dat',
 './data/atuswho_2005.dat',
 './data/atuswho_2006.dat',
 './data/atuswho_2007.dat',
 './data/atuswho_2008.dat',
 './data/atuswho_2009.dat',
 './data/atuswho_2010.dat',
 './data/atuswho_2011.dat',
 './data/atuswho_2012.dat',
 './data/atuswho_2013.dat',
 './data/atuswho_2014.dat',
 './data/atuswho_2015.dat',
 './data/atuswho_2017.dat',
 './data/atuswho_2018.dat',
 './data/atuswho_2019.dat',
 './data/atuswho_2020.dat',
 './data/atuswho_2021.dat',
 './data/atuswho_2022.dat',
 './data/atuswho_2023.dat',
 './data/atuswho_2024.dat',
 './data/atuswho_2003/atuswho_2003.dat',
 './data/atuswho_2016/atuswho_2016.dat']

In [32]:
(who_column_summary :=
 pipe(who_paths,
     lambda L: [pl.read_csv(p)
                  .with_columns(file = pl.lit(p.replace('\\','/').split('/')[-1]))
                  .head(1) 
                  .unpivot(index='file', 
                           variable_name='Column')
                  .drop('value')
                  .with_columns(pl.lit(1).alias('ones'))
                  .pivot(index = 'Column', 
                         on='file', 
                         values='ones') 
                for p in L 
                ],
     lambda L: reduce(join_next, L).sort('Column').fill_null(0),
     lambda df: df.with_columns(all = pl.reduce(mul, cs.starts_with(type)), 
                                count = pl.reduce(add, cs.starts_with(type))
                               ),
     )
)

Column,atuswho_2004.dat,atuswho_2005.dat,atuswho_2006.dat,atuswho_2007.dat,atuswho_2008.dat,atuswho_2009.dat,atuswho_2010.dat,atuswho_2011.dat,atuswho_2012.dat,atuswho_2013.dat,atuswho_2014.dat,atuswho_2015.dat,atuswho_2017.dat,atuswho_2018.dat,atuswho_2019.dat,atuswho_2020.dat,atuswho_2021.dat,atuswho_2022.dat,atuswho_2023.dat,atuswho_2024.dat,atuswho_2003.dat,atuswho_2016.dat,all,count
str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""TRWHONA""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22
"""TUACTIVITY_N""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22
"""TUCASEID""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22
"""TULINENO""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22
"""TUWHO_CODE""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22


In [33]:
#  show  only columns that are mismatched between the different years.
# there ar eno missing columns, and basically every column in this table is needed.
who_column_summary.filter(pl.col('all')==0)

Column,atuswho_2004.dat,atuswho_2005.dat,atuswho_2006.dat,atuswho_2007.dat,atuswho_2008.dat,atuswho_2009.dat,atuswho_2010.dat,atuswho_2011.dat,atuswho_2012.dat,atuswho_2013.dat,atuswho_2014.dat,atuswho_2015.dat,atuswho_2017.dat,atuswho_2018.dat,atuswho_2019.dat,atuswho_2020.dat,atuswho_2021.dat,atuswho_2022.dat,atuswho_2023.dat,atuswho_2024.dat,atuswho_2003.dat,atuswho_2016.dat,all,count
str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32


### ROSTER PRE-UNION COLUMN TEST

In [34]:
type = 'atusrost'
(rost_paths :=
 glob('./data/atusrost*.dat') + glob('./data/*/atusrost*.dat')
)

['./data/atusrost_2003.dat',
 './data/atusrost_2004.dat',
 './data/atusrost_2005.dat',
 './data/atusrost_2006.dat',
 './data/atusrost_2007.dat',
 './data/atusrost_2008.dat',
 './data/atusrost_2009.dat',
 './data/atusrost_2010.dat',
 './data/atusrost_2011.dat',
 './data/atusrost_2012.dat',
 './data/atusrost_2013.dat',
 './data/atusrost_2014.dat',
 './data/atusrost_2015.dat',
 './data/atusrost_2017.dat',
 './data/atusrost_2018.dat',
 './data/atusrost_2019.dat',
 './data/atusrost_2020.dat',
 './data/atusrost_2021.dat',
 './data/atusrost_2022.dat',
 './data/atusrost_2023.dat',
 './data/atusrost_2024.dat',
 './data/atusrost_2016/atusrost_2016.dat']

In [35]:
(rost_column_summary :=
 pipe(rost_paths,
     lambda L: [pl.read_csv(p)
                  .with_columns(file = pl.lit(p.replace('\\','/').split('/')[-1]))
                  .head(1) 
                  .unpivot(index='file', 
                           variable_name='Column')
                  .drop('value')
                  .with_columns(pl.lit(1).alias('ones'))
                  .pivot(index = 'Column', 
                         on='file', 
                         values='ones') 
                for p in L 
                ],
     lambda L: reduce(join_next, L).sort('Column').fill_null(0),
     lambda df: df.with_columns(all = pl.reduce(mul, cs.starts_with(type)), 
                                count = pl.reduce(add, cs.starts_with(type))
                               ),
     )
)

Column,atusrost_2003.dat,atusrost_2004.dat,atusrost_2005.dat,atusrost_2006.dat,atusrost_2007.dat,atusrost_2008.dat,atusrost_2009.dat,atusrost_2010.dat,atusrost_2011.dat,atusrost_2012.dat,atusrost_2013.dat,atusrost_2014.dat,atusrost_2015.dat,atusrost_2017.dat,atusrost_2018.dat,atusrost_2019.dat,atusrost_2020.dat,atusrost_2021.dat,atusrost_2022.dat,atusrost_2023.dat,atusrost_2024.dat,atusrost_2016.dat,all,count
str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""TEAGE""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22
"""TERRP""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22
"""TESEX""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22
"""TUCASEID""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22
"""TULINENO""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22
"""TURRP""",1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
"""TXAGE""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22
"""TXRRP""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22
"""TXSEX""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22


In [36]:
#  show  only columns that are mismatched between the different years.
# turrp won't actually be needed, so we will allow the next stage of the process to automatically remove it prior to unioning
rost_column_summary.filter(pl.col('all')==0)

Column,atusrost_2003.dat,atusrost_2004.dat,atusrost_2005.dat,atusrost_2006.dat,atusrost_2007.dat,atusrost_2008.dat,atusrost_2009.dat,atusrost_2010.dat,atusrost_2011.dat,atusrost_2012.dat,atusrost_2013.dat,atusrost_2014.dat,atusrost_2015.dat,atusrost_2017.dat,atusrost_2018.dat,atusrost_2019.dat,atusrost_2020.dat,atusrost_2021.dat,atusrost_2022.dat,atusrost_2023.dat,atusrost_2024.dat,atusrost_2016.dat,all,count
str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""TURRP""",1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2


### RESPONDENT PRE-UNION COLUMN TEST

In [37]:
type = 'atusresp'
(resp_paths :=
 glob('./data/atusresp*.dat') + glob('./data/*/atusresp*.dat')
)

['./data/atusresp_2003.dat',
 './data/atusresp_2004.dat',
 './data/atusresp_2005.dat',
 './data/atusresp_2006.dat',
 './data/atusresp_2007.dat',
 './data/atusresp_2008.dat',
 './data/atusresp_2009.dat',
 './data/atusresp_2010.dat',
 './data/atusresp_2011.dat',
 './data/atusresp_2012.dat',
 './data/atusresp_2013.dat',
 './data/atusresp_2014.dat',
 './data/atusresp_2015.dat',
 './data/atusresp_2017.dat',
 './data/atusresp_2018.dat',
 './data/atusresp_2019.dat',
 './data/atusresp_2020.dat',
 './data/atusresp_2021.dat',
 './data/atusresp_2022.dat',
 './data/atusresp_2023.dat',
 './data/atusresp_2024.dat',
 './data/atusresp_2016/atusresp_2016.dat']

In [38]:
(resp_column_summary :=
 pipe(resp_paths,
     lambda L: [pl.read_csv(p,n_rows=100,infer_schema_length=10000)
                  .with_columns(file = pl.lit(p.replace('\\','/').split('/')[-1]))
                  .head(1) 
                  .unpivot(index='file', 
                           variable_name='Column')
                  .drop('value')
                  .with_columns(pl.lit(1).alias('ones'))
                  .pivot(index = 'Column', 
                         on='file', 
                         values='ones') 
                for p in L 
                ],
     lambda L: reduce(join_next, L).sort('Column').fill_null(0),
     lambda df: df.with_columns(all = pl.reduce(mul, cs.starts_with(type)), 
                                count = pl.reduce(add, cs.starts_with(type))
                               ),
     )
)

Column,atusresp_2003.dat,atusresp_2004.dat,atusresp_2005.dat,atusresp_2006.dat,atusresp_2007.dat,atusresp_2008.dat,atusresp_2009.dat,atusresp_2010.dat,atusresp_2011.dat,atusresp_2012.dat,atusresp_2013.dat,atusresp_2014.dat,atusresp_2015.dat,atusresp_2017.dat,atusresp_2018.dat,atusresp_2019.dat,atusresp_2020.dat,atusresp_2021.dat,atusresp_2022.dat,atusresp_2023.dat,atusresp_2024.dat,atusresp_2016.dat,all,count
str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""TEABSRSN""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22
"""TEERN""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22
"""TEERNH1O""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22
"""TEERNH2""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22
"""TEERNHRO""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,22
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""TXTHH""",0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,21
"""TXTNOHH""",0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,21
"""TXTO""",0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,21


In [39]:
#  show  only columns that are mismatched between the different years. 
# in effect none of the columns here are needed except for the weight columns which will be delt with in the next part of the project
resp_column_summary.filter(pl.col('all')==0)

Column,atusresp_2003.dat,atusresp_2004.dat,atusresp_2005.dat,atusresp_2006.dat,atusresp_2007.dat,atusresp_2008.dat,atusresp_2009.dat,atusresp_2010.dat,atusresp_2011.dat,atusresp_2012.dat,atusresp_2013.dat,atusresp_2014.dat,atusresp_2015.dat,atusresp_2017.dat,atusresp_2018.dat,atusresp_2019.dat,atusresp_2020.dat,atusresp_2021.dat,atusresp_2022.dat,atusresp_2023.dat,atusresp_2024.dat,atusresp_2016.dat,all,count
str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""TREMODR""",0,0,0,1,1,1,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,1,0,8
"""TRLVMODR""",0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,4
"""TRNUMHOU""",0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,20
"""TRTALONE_WK""",0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,15
"""TRTCCC_WK""",0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,15
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""TXTHH""",0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,21
"""TXTNOHH""",0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,21
"""TXTO""",0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,21
